In [25]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [30]:
pwd

'/Users/takizawakeiya/Desktop/python/git_test/statistical-analysis-toolkit'

In [36]:
cd src

/Users/takizawakeiya/Desktop/python/git_test/statistical-analysis-toolkit/src


/Users/takizawakeiya/Desktop/python/Takizawa-venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
ls

README.md  doc/       notebook/  sample/    src/       test/


In [38]:
from distribution_moments import (
    DiscreteUniformMoments,
    ContinuousUniformMoments,
    BernoulliMoments
)

In [41]:
dum = DiscreteUniformMoments(0,2)
print(dum.mean())
print(dum.var())

1.0
0.6666666666666666


In [42]:
cum = ContinuousUniformMoments(0,2)
print(cum.mean())
print(cum.var())

1.0
0.3333333333333333


In [44]:
ber = BernoulliMoments(0.2)
print(ber.mean())
print(ber.var())

0.2
0.16000000000000003
